In [6]:
import torch
from torch.optim import SGD

from avalanche.benchmarks.datasets import CORe50Dataset
from avalanche.benchmarks.utils import AvalancheDataset
from avalanche.benchmarks.generators import nc_scenario
from torchvision.transforms import (
    ToTensor,
    Normalize,
    Compose,
    RandomHorizontalFlip,
)

from avalanche.models import IcarlNet, make_icarl_net, initialize_icarl_net
from avalanche.training import ICaRL
from avalanche.logging import InteractiveLogger
from avalanche.logging.text_logging import TextLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import (
    accuracy_metrics,
    forgetting_metrics,
    cpu_usage_metrics,
    gpu_usage_metrics,
    ram_usage_metrics,
    timing_metrics,
)

In [8]:
# Config
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Eval_metics
eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    gpu_usage_metrics(gpu_id=0,experience=True),
    ram_usage_metrics(experience=True),
    timing_metrics(experience=True),
    loggers=[InteractiveLogger(),
                TextLogger(open('icarl_results.txt', 'w'))]
)

# model
#model = SimpleCNN(num_classes=50)
model: IcarlNet = make_icarl_net(num_classes=50)
model.apply(initialize_icarl_net)

# Prepare for training & testing
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
#criterion = CrossEntropyLoss()

# CL Benchmark Creation
train_set = CORe50Dataset(train=True, mini=True)
test_set = CORe50Dataset(train=False, mini=True)

normalize = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

transforms_group = dict(
    eval=(
        Compose(
            [
                ToTensor(), 
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ]
        ),
        None,
    ),
    train=(
        Compose(
            [
                ToTensor(), 
                RandomHorizontalFlip(), 
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ]
        ),
        None,
    ),
)

train_set = AvalancheDataset(
    train_set,
    transform_groups=transforms_group,
    initial_transform_group="train",
)
test_set = AvalancheDataset(
    test_set,
    transform_groups=transforms_group,
    initial_transform_group="eval",
)

scenario = nc_scenario(
    train_dataset=train_set,
    test_dataset=test_set,
    n_experiences=10, # For each experience, the number of new classes is fiexed.
    task_labels=False
)

E:\python\CL\avalanche_root\avalanche\training\plugins\evaluation.py:81: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  warnings.warn(


Loading labels...
Loading LUP...
Loading labels names...
Files already downloaded and verified
Loading labels...
Loading LUP...
Loading labels names...
Files already downloaded and verified


In [9]:
# Continual learning strategy
cl_strategy = ICaRL(
    model.feature_extractor,
    model.classifier, 
    optimizer, 
    memory_size=250,
    buffer_transform=False,
    fixed_memory=True,
    train_mb_size=128, train_epochs=5,
    eval_mb_size=128, device=device,
    evaluator=eval_plugin)

# train and test loop over the stream of experiences
results = []
for train_exp in scenario.train_stream:
    cl_strategy.train(train_exp)
    results.append(cl_strategy.eval(scenario.test_stream))
    break

-- >> Start of training phase << --
 13%|█▎        | 12/94 [00:03<00:13,  5.99it/s]